In [1]:
!pip install torch quanto transformers accelerate bitsandbytes evaluate datasets --upgrade --quiet

## Quanto Method

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModel, QuantoConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

quantization_config = QuantoConfig(weights="int8",activations=None)


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",device_map="auto",quantization_config=quantization_config)




tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [3]:

def print_param_dtype(model):
    for name, param in model.named_parameters():
        print(f"{name} is loaded in {param.dtype}")

print_param_dtype(model)

model.embed_tokens.weight is loaded in torch.float32
model.layers.0.self_attn.q_proj.weight is loaded in torch.float32
model.layers.0.self_attn.k_proj.weight is loaded in torch.float32
model.layers.0.self_attn.v_proj.weight is loaded in torch.float32
model.layers.0.self_attn.o_proj.weight is loaded in torch.float32
model.layers.0.mlp.gate_proj.weight is loaded in torch.float32
model.layers.0.mlp.up_proj.weight is loaded in torch.float32
model.layers.0.mlp.down_proj.weight is loaded in torch.float32
model.layers.0.input_layernorm.weight is loaded in torch.float32
model.layers.0.post_attention_layernorm.weight is loaded in torch.float32
model.layers.1.self_attn.q_proj.weight is loaded in torch.float32
model.layers.1.self_attn.k_proj.weight is loaded in torch.float32
model.layers.1.self_attn.v_proj.weight is loaded in torch.float32
model.layers.1.self_attn.o_proj.weight is loaded in torch.float32
model.layers.1.mlp.gate_proj.weight is loaded in torch.float32
model.layers.1.mlp.up_proj.wei

In [4]:
model.model.layers[0].self_attn.q_proj.weight

QTensor(tensor([[  0,   5,   0,  ...,  26,   0,  -5],
        [  0,   0,   0,  ...,   1,   0,   2],
        [  0,  -1,   0,  ..., -11,   0,   0],
        ...,
        [  0,  -3,   0,  ...,   0,   0,   1],
        [  0,  -3,   0,  ...,  -5,   0,   1],
        [  0,   3,   0,  ...,   0,   0,  -1]], device='cuda:0',
       dtype=torch.int8), scale=tensor([[0.0001],
        [0.0002],
        [0.0005],
        ...,
        [0.0005],
        [0.0006],
        [0.0005]], device='cuda:0'), public_dtype=torch.float32)

In [6]:
def generate(model,text):
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, do_sample=True, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [21]:
out =generate(model,"who is mohamed abu trika?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [22]:
print(out)

who is mohamed abu trika?

Mohamed Abou Trika (Arabic: محمد ابو تريكة, born on February 7, 1976) is a retired Egyptian international footballer. He is commonly known as Trézéguet. He played as a forward.

Abou Trika began his professional career with Egyptian club Tersana in 1996


## bitsandbytes method

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#quantization_config = QuantoConfig(weights="int8",activations=None)


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model2 = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",device_map="auto",load_in_8bit=True,torch_dtype=torch.float32)




The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
model2.model.layers[0].self_attn.q_proj.weight

Parameter containing:
Parameter(Int8Params([[  0,   5,   0,  ...,  26,   0,  -5],
            [  0,   0,   0,  ...,   1,   0,   2],
            [  0,  -1,   0,  ..., -11,   0,   0],
            ...,
            [  0,  -3,   0,  ...,   0,   0,   1],
            [  0,  -3,   0,  ...,  -5,   0,   1],
            [  0,   3,   0,  ...,   0,   0,  -1]], device='cuda:0',
           dtype=torch.int8))

In [16]:
def generate(model,text):
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, do_sample=True, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [32]:
out2 = generate(model2,"who is mohamed abu trika?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [33]:
print(out2)

who is mohamed abu trika?
Mohamed Abu Trika is an Egyptian professional footballer who currently plays as a left winger for Egyptian Premier League club Tala El Gaish. He has also represented the Egyptian national team at various levels, including the Egypt U23 team, and was part of the squad that won the gold medal at the 2012 Summer Olympics. Abu Trika started his professional career in Egypt, playing for Haras El H


In [34]:
def calculate_perplexity(model, text):
    # Encode the text
    encodings = tokenizer(text, return_tensors='pt').to(device)

    input_ids = encodings.input_ids
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

    # Loss calculation
    neg_log_likelihood = outputs.loss



    # Perplexity calculation
    ppl = torch.exp(neg_log_likelihood)

    return ppl



In [35]:


ppl1 = calculate_perplexity(model, out)
ppl2 = calculate_perplexity(model, out2)

print(f"quanto perplexity:  {ppl1.item():.2f}")
print(f"bitsandbytes perplexity:  {ppl2.item():.2f}")


quanto perplexity:  2.54
bitsandbytes perplexity:  2.99


In [ ]:
from huggingface_hub import notebook_login ,create_repo

notebook_login()

In [ ]:
create_repo("samehfarouk/Mistral-7B-Instruct-v0.2_int8", exist_ok=True)

In [ ]:
model.push_to_hub("samehfarouk/Mistral-7B-Instruct-v0.2_int8")
tokenizer.push_to_hub("samehfarouk/Mistral-7B-Instruct-v0.2_int8")